In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/My Drive/kifiya5/complaints.csv")

/tmp/ipython-input-3-1083509708.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/My Drive/kifiya5/complaints.csv")


In [4]:
print("\nFirst 5 rows:")
print(df.head())


First 5 rows:
  Date received                                            Product  \
0    2025-06-20  Credit reporting or other personal consumer re...   
1    2025-06-20                                    Debt collection   
2    2025-06-20  Credit reporting or other personal consumer re...   
3    2025-06-20  Credit reporting or other personal consumer re...   
4    2025-06-20  Credit reporting or other personal consumer re...   

               Sub-product                                 Issue  \
0         Credit reporting  Incorrect information on your report   
1  Telecommunications debt     Attempts to collect debt not owed   
2         Credit reporting           Improper use of your report   
3         Credit reporting           Improper use of your report   
4         Credit reporting  Incorrect information on your report   

                                       Sub-issue Consumer complaint narrative  \
0            Information belongs to someone else                          

In [7]:
# prompt: write code to which list all column in df

print("\nColumns in the DataFrame:")
df.columns


Columns in the DataFrame:


Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [9]:
# Define the list of specified products (using partial matching for flexibility)
specified_products = ['Credit card', 'Personal loan', 'Buy Now, Pay Later', 'Savings account', 'Money transfer']

# Filter the dataset using Product and Sub-product columns
df_filtered = df[df['Product'].str.contains('|'.join(specified_products), case=False, na=False) |
                 df['Sub-product'].str.contains('|'.join(specified_products), case=False, na=False)]

# Verify the filtered dataset
print(f"\nNumber of records after filtering by product: {len(df_filtered)}")
print("\nDistribution of filtered products:")
print(df_filtered['Product'].value_counts())
print("\nDistribution of filtered sub-products:")
print(df_filtered['Sub-product'].value_counts())


Number of records after filtering by product: 1089873

Distribution of filtered products:
Product
Checking or savings account                                291178
Credit card                                                226686
Credit card or prepaid card                                206369
Debt collection                                            162753
Money transfer, virtual currency, or money service         145066
Payday loan, title loan, or personal loan                   30641
Payday loan, title loan, personal loan, or advance loan     16514
Money transfers                                              5354
Bank account or service                                      5294
Virtual currency                                               18
Name: count, dtype: int64

Distribution of filtered sub-products:
Sub-product
General-purpose credit card or charge card          281931
Checking account                                    234253
Credit card debt                             

In [10]:
# Remove records with empty narratives
df_filtered = df_filtered[df_filtered['Consumer complaint narrative'].notnull()]
df_filtered = df_filtered[df_filtered['Consumer complaint narrative'].str.strip() != '']

print(f"\nNumber of records after removing empty narratives: {len(df_filtered)}")


Number of records after removing empty narratives: 527570


In [12]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk


In [16]:
# Download NLTK resources (run once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [17]:

# Define a function to clean text
def clean_narrative(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove boilerplate text (example phrases based on CFPB narrative patterns)
    boilerplate_phrases = [
        r'i am writing to file a complaint',
        r'this is regarding a complaint',
        r'please address this issue',
        r'consumer complaint',
        r'i would like to report'
    ]
    for phrase in boilerplate_phrases:
        text = re.sub(phrase, '', text, flags=re.IGNORECASE)

    # Remove extra whitespace
    text = ' '.join(text.split())

    # Optional: Remove stopwords (only if beneficial for the RAG model)
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])

    return text


In [18]:

# Apply cleaning to the narratives
df_filtered['Cleaned_Narrative'] = df_filtered['Consumer complaint narrative'].apply(clean_narrative)


In [19]:
# Display a sample of cleaned narratives
print("\nSample of cleaned narratives:")
print(df_filtered[['Consumer complaint narrative', 'Cleaned_Narrative']].head())


Sample of cleaned narratives:
                            Consumer complaint narrative  \
12237  A XXXX XXXX card was opened under my name by a...   
12532  I made the mistake of using my wellsfargo debi...   
13280  Dear CFPB, I have a secured credit card with c...   
13506  I have a Citi rewards cards. The credit balanc...   
13955  b'I am writing to dispute the following charge...   

                                       Cleaned_Narrative  
12237  xxxx xxxx card opened name fraudster received ...  
12532  made mistake using wellsfargo debit card depsi...  
13280  dear cfpb secured credit card citibank changed...  
13506  citi rewards cards credit balance issued recen...  
13955  bi writing dispute following charges citi cred...  


In [21]:
# prompt: # Save the preprocessed dataset
# df_filtered[['Complaint ID', 'Product', 'Sub-product', 'Cleaned_Narrative']].to_csv('preprocessed_complaints.csv', index=False)
# print("\nPreprocessed dataset saved as 'preprocessed_complaints.csv'")

df_filtered[['Complaint ID', 'Product', 'Sub-product', 'Cleaned_Narrative']].to_csv('/content/drive/My Drive/kifiya5/preprocessed_complaints.csv', index=False)
print("\nPreprocessed dataset saved as '/content/drive/My Drive/kifiya5/preprocessed_complaints.csv'")



Preprocessed dataset saved as '/content/drive/My Drive/kifiya5/preprocessed_complaints.csv'
